### Урок 7. Selenium в Python

#### 1) Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных (от кого, дата отправки, тема письма, текст письма полный)
Логин тестового ящика: study.ai_172@mail.ru<br>
Пароль тестового ящика: NextPassword172

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import exceptions
from pymongo import MongoClient
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

def _parse_element(element, css_selector):
    result = WebDriverWait(element, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, css_selector))).text
    return result


def parse_email(element):
    item = {}

    item['from_name'] = _parse_element(
        element, 'span[class~="ns-view-message-head-sender-name"]')
    item['from_email'] = _parse_element(
        element, 'span[class~="mail-Message-Sender-Email"]')
    item['date'] = _parse_element(element,
                                  'div[class~="ns-view-message-head-date"]')
    item['subject'] = _parse_element(
        element, 'div[class~="mail-Message-Toolbar-Subject"]')
    item['text_messege'] = _parse_element(
        element, 'div.mail-Message-Body-Content')

    return item


MONGO_URI = 'mongodb://127.0.0.1:27017/'
MONGO_DATABASE = 'mail_db'

client = MongoClient(MONGO_URI)
mongo_base = client[MONGO_DATABASE]
collection = mongo_base['messeges']

chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome('/usr/local/bin/chromedriver')

url = 'https://mail.ru/'
title_site = 'Mail.ru:'

driver.get(url)

assert title_site in driver.title

driver.title

field_login = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, 'mailbox:login-input')))
field_login.send_keys('study.ai_172')
field_login.send_keys(Keys.ENTER)
field_passwd = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, 'mailbox:password-input')))
field_passwd.send_keys('NextPassword172')
field_passwd.send_keys(Keys.ENTER)

first_messege = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable(
        (By.XPATH, "//a[@class='llc js-tooltip-direction_letter-bottom js-letter-list-item llc_normal'][1]")
    )
)
first_messege.click()
while True:
    try:

        mouse = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable(
                (By.XPATH, "//span[@class='button2 button2_has-ico button2_arrow-down button2_pure button2_short button2_ico-text-top button2_hover-support js-shortcut']")))
        ActionChains(driver).move_to_element(mouse).perform()

        ActionChains(driver).move_to_element(mouse).perform()
        mouse.click()

        collection.insert_one(parse_email(driver))


    except exceptions.TimeoutException:
        print('E-mails are over')
        break

driver.quit()

#### 2) Написать программу, которая собирает «Хиты продаж» с сайта техники mvideo и складывает данные в БД. Магазины можно выбрать свои. Главный критерий выбора: динамически загружаемые товары

In [ ]:
from selenium import webdriver
from pymongo import MongoClient
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import exceptions

MONGO_URI = 'mongodb://127.0.0.1:27017/'
MONGO_DATABASE = 'mvideo_db'

client = MongoClient(MONGO_URI)
mongo_base = client[MONGO_DATABASE]
collection = mongo_base['bestsellers']

chrome_options = Options()
# chrome_options.add_argument("--headless")

driver = webdriver.Chrome('/usr/local/bin/chromedriver', options=chrome_options)

url = 'https://www.mvideo.ru'
title_site = 'М.Видео'

driver.get(url)

assert title_site in driver.title

try:
    hits_paid = driver.find_element_by_xpath(
        '//div[contains(text(),"Хиты продаж")]/ancestor::div[@data-init="gtm-push-products"]'
    )
except exceptions.NoSuchElementException:
    print('Bestsellers has not been found')

while True:
    try:
        next_button = WebDriverWait(hits_paid, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'a[class="next-btn sel-hits-button-next"]')
            )
        )
        next_button.click()


    except exceptions.TimeoutException:
        print('Сбор данных окончен')
        break

goods = hits_paid.find_elements_by_css_selector('li.gallery-list-item')

item = {}
for good in goods:
    item['title'] = good.find_element_by_css_selector(
        'a.sel-product-tile-title') \
        .get_attribute('innerHTML')

    item['good_link'] = good.find_element_by_css_selector(
        'a.sel-product-tile-title') \
        .get_attribute('href')

    item['price'] = float(
        good.find_element_by_css_selector(
            'div.c-pdp-price__current').get_attribute('innerHTML').replace(
            '&nbsp;', '').replace('¤', ''))

    item['image_link'] = good.find_element_by_css_selector(
        'img[class="lazy product-tile-picture__image"]') \
        .get_attribute('src')

    collection.update_one({'good_link': item['good_link']}, {'$set': item},
                          upsert=True)
driver.quit()
